# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [212]:
# import libraries
import nltk 
nltk.download(['wordnet', 'punkt', 'stopwords'])
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
import os
import re
import pickle


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DIBARRAG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DIBARRAG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DIBARRAG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [213]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse_table", engine)

X = df['message']  # Message Column
Y = df.iloc[:, 4:] # Classification label




In [214]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response.
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class.
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [ ]:
# check if all '2' have been replace with '1'
df.groupby('related').count()

In [ ]:
# Extract X and y variables from the data for the modelling
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [197]:
 # Replace all urls with a urlplaceholder string
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
  # tokenize text
  # initiate lemmatizer
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
  # iterate through each token
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [198]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False
# Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [199]:
def ML_pipeline(clf = AdaBoostClassifier()):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    
    return pipeline




### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [200]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

modelo = ML_pipeline()
modelo.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001676E9E3CA0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [ ]:
##print(y_test.shape,type(y_test))
#Checkign shape & type of y_pred

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [201]:
y_prediction_train = modelo.predict(X_train)
y_prediction_test = modelo.predict(X_test)

In [202]:
# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.88      5063
               request       0.75      0.54      0.63      1125
                 offer       0.08      0.03      0.05        29
           aid_related       0.77      0.58      0.66      2757
          medical_help       0.61      0.24      0.35       523
      medical_products       0.67      0.33      0.44       345
     search_and_rescue       0.58      0.15      0.24       200
              security       0.14      0.03      0.05       111
              military       0.59      0.34      0.43       202
                 water       0.76      0.67      0.71       448
                  food       0.82      0.69      0.75       771
               shelter       0.78      0.54      0.64       617
              clothing       0.70      0.41      0.51       116
                 money       0.61      0.34      0.44       136
        missing_people       0.53      

C:\Users\DIBARRAG\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DIBARRAG\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [203]:
# get list of model parameters
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb_transformer', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__

In [204]:
# In view of that, only a few parameters are chosen to demonstrate the concept.
# To obtain a highly optimized model, we need to increase the number of parameters.

    #parameters = {
     #   'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
      #  'clf__n_estimators': [50, 100, 200],
       # 'clf__min_samples_split': [2, 3, 4]
   # }

modelo_1 = ML_pipeline()
parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [10, 20]
}


cv = GridSearchCV(modelo_1, param_grid=parameters,cv=5, n_jobs=-1, verbose=3)



In [205]:
cv

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000001676E9E3CA0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             

In [206]:
# Find best parameters
cv.fit(X_train, y_train)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000001676E9E3CA0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [207]:
# Get the prediction values from the grid search cross validator
y_pred_test = cv.predict(X_test)
y_pred_train = cv.predict(X_train)

In [208]:
# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.88      5063
               request       0.73      0.50      0.59      1125
                 offer       0.00      0.00      0.00        29
           aid_related       0.75      0.54      0.63      2757
          medical_help       0.58      0.20      0.30       523
      medical_products       0.74      0.30      0.43       345
     search_and_rescue       0.67      0.15      0.24       200
              security       0.10      0.02      0.03       111
              military       0.69      0.25      0.36       202
                 water       0.78      0.66      0.71       448
                  food       0.82      0.67      0.74       771
               shelter       0.79      0.48      0.59       617
              clothing       0.75      0.42      0.54       116
                 money       0.65      0.35      0.45       136
        missing_people       0.79      

C:\Users\DIBARRAG\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DIBARRAG\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [209]:
# try other model -> RandomForestClassifier
model_rf = ML_pipeline(clf = RandomForestClassifier())
model_rf.fit(X_train, y_train)
y_pred_test_rf = model_rf.predict(X_test)

In [210]:
print(classification_report(y_test.values, y_pred_test_rf, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5063
               request       0.89      0.43      0.58      1125
                 offer       0.00      0.00      0.00        29
           aid_related       0.80      0.60      0.68      2757
          medical_help       0.63      0.03      0.06       523
      medical_products       0.78      0.06      0.11       345
     search_and_rescue       0.86      0.03      0.06       200
              security       0.00      0.00      0.00       111
              military       0.92      0.05      0.10       202
                 water       0.91      0.27      0.41       448
                  food       0.90      0.38      0.53       771
               shelter       0.82      0.19      0.31       617
              clothing       0.73      0.07      0.13       116
                 money       0.80      0.03      0.06       136
        missing_people       0.40      

C:\Users\DIBARRAG\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DIBARRAG\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DIBARRAG\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [211]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(model_rf, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.